In [29]:
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [30]:
sample = """<h1>Title Goes Here</h1>
<b>Bolded Text</b>
<i>Italicized Text</i>
<img src="this should all be gone"/>
<a href="this will be gone, too">But this will still be here!</a>
I run. He ran. She is running. Will they stop running?
I talked. She was talking. They talked to them about running. Who ran to the talking runner?
[Some text we don't want to keep is in here]
¡Sebastián, Nicolás, Alejandro and Jéronimo are going to the store tomorrow morning!
something... is! wrong() with.,; this :: sentence.
I can't do this anymore. I didn't know them. Why couldn't you have dinner at the restaurant?
My favorite movie franchises, in order: Indiana Jones; Marvel Cinematic Universe; Star Wars; Back to the Future; Harry Potter.
Don't do it.... Just don't. Billy! I know what you're doing. This is a great little house you've got here.
[This is some other unwanted text]
John: "Well, well, well."
James: "There, there. There, there."
&nbsp;&nbsp;
There are a lot of reasons not to do this. There are 101 reasons not to do it. 1000000 reasons, actually.
I have to go get 2 tutus from 2 different stores, too.
22    45   1067   445
{{Here is some stuff inside of double curly braces.}}
{Here is more stuff in single curly braces.}
[DELETE]
</body>
</html>"""

In [31]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

sample = denoise_text(sample)
#sample

In [32]:
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

sample = replace_contractions(sample)
#sample

In [38]:
#words = nltk.word_tokenize(sample)

sentences = nltk.sent_tokenize(sample)
sentences

['Title Goes Here\nBolded Text\nItalicized Text\n\nBut this will still be here!',
 'I run.',
 'He ran.',
 'She is running.',
 'Will they stop running?',
 'I talked.',
 'She was talking.',
 'They talked to them about running.',
 'Who ran to the talking runner?',
 '¡Sebastián, Nicolás, Alejandro and Jéronimo are going to the store tomorrow morning!',
 'something... is!',
 'wrong() with.,; this :: sentence.',
 'I cannot do this anymore.',
 'I did not know them.',
 'Why could not you have dinner at the restaurant?',
 'My favorite movie franchises, in order: Indiana Jones; Marvel Cinematic Universe; Star Wars; Back to the Future; Harry Potter.',
 'do not do it.... Just do not.',
 'Billy!',
 'I know what you are doing.',
 'This is a great little house you have got here.',
 'John: "Well, well, well."',
 'James: "There, there.',
 'There, there."',
 'There are a lot of reasons not to do this.',
 'There are 101 reasons not to do it.',
 '1000000 reasons, actually.',
 'I have to go get 2 tutus fro

In [39]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words



In [42]:
sentences = normalize(sentences)
sentences

['title goes here\nbolded text\nitalicized text\n\nbut this will still be here',
 'i run',
 'he ran',
 'she is running',
 'will they stop running',
 'i talked',
 'she was talking',
 'they talked to them about running',
 'who ran to the talking runner',
 'sebastian nicolas alejandro and jeronimo are going to the store tomorrow morning',
 'something is',
 'wrong with this  sentence',
 'i cannot do this anymore',
 'i did not know them',
 'why could not you have dinner at the restaurant',
 'my favorite movie franchises in order indiana jones marvel cinematic universe star wars back to the future harry potter',
 'do not do it just do not',
 'billy',
 'i know what you are doing',
 'this is a great little house you have got here',
 'john well well well',
 'james there there',
 'there there',
 'there are a lot of reasons not to do this',
 'there are 101 reasons not to do it',
 '1000000 reasons actually',
 'i have to go get 2 tutus from 2 different stores too',
 '22    45   1067   445\nhere is 

In [34]:
def stem_and_lemmatize(sentences):
    stems = stem_words(sentences)
    lemmas = lemmatize_verbs(sentences)
    return stems, lemmas

stems, lemmas = stem_and_lemmatize(sentences)
stems
lemmas

['Title Goes Here\nBolded Text\nItalicized Text\n\nBut this will still be here!',
 'I run.',
 'He ran.',
 'She is running.',
 'Will they stop running?',
 'I talked.',
 'She was talking.',
 'They talked to them about running.',
 'Who ran to the talking runner?',
 '¡Sebastián, Nicolás, Alejandro and Jéronimo are going to the store tomorrow morning!',
 'something... is!',
 'wrong() with.,; this :: sentence.',
 'I cannot do this anymore.',
 'I did not know them.',
 'Why could not you have dinner at the restaurant?',
 'My favorite movie franchises, in order: Indiana Jones; Marvel Cinematic Universe; Star Wars; Back to the Future; Harry Potter.',
 'do not do it.... Just do not.',
 'Billy!',
 'I know what you are doing.',
 'This is a great little house you have got here.',
 'John: "Well, well, well."',
 'James: "There, there.',
 'There, there."',
 'There are a lot of reasons not to do this.',
 'There are 101 reasons not to do it.',
 '1000000 reasons, actually.',
 'I have to go get 2 tutus fro